In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import expected_returns, risk_models, BlackLittermanModel
import yfinance as yf

In [26]:
data = {}
start_date = '2020-01-01'
end_date = '2020-12-31'

In [27]:
data['VTI'] = yf.download('VTI', period='max')
data['VTV'] = yf.download('VTV', period='max')
data['VOE'] = yf.download('VOE', period='max')
data['VBR'] = yf.download('VBR', period='max')
data['GBIL'] = yf.download('GBIL', period='max')
data['JPST'] = yf.download('JPST', period='max')
data['VNQ'] = yf.download('VNQ', period='max')
data['VPU'] = yf.download('VPU', period='max')
#data['BITO'] = yf.download('BITO', period='max')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [51]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import time #helper libraries

In [30]:
scalar = MinMaxScaler(feature_range=(0, 1))

In [31]:
data['VTI'] = scalar.fit_transform(data['VTI'].dropna().values.reshape(-1, 1))
data['VTV'] = scalar.fit_transform(data['VTV'].dropna().values.reshape(-1, 1))
data['VOE'] = scalar.fit_transform(data['VOE'].dropna().values.reshape(-1, 1))
data['VBR'] = scalar.fit_transform(data['VBR'].dropna().values.reshape(-1, 1))
data['GBIL'] = scalar.fit_transform(data['GBIL'].dropna().values.reshape(-1, 1))
data['JPST'] = scalar.fit_transform(data['JPST'].dropna().values.reshape(-1, 1))
data['VNQ'] = scalar.fit_transform(data['VNQ'].dropna().values.reshape(-1, 1))
data['VPU'] = scalar.fit_transform(data['VPU'].dropna().values.reshape(-1, 1))
#data['BITO'] = scalar.fit_transform(data['BITO'].dropna().values.reshape(-1, 1))

In [33]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [34]:
trainingData = {}
testData = {}

In [35]:
for ticker in data:
    trainingData[ticker], testData[ticker] = train_test_split(data[ticker], test_size=0.2)

In [38]:
trainX, trainY = {}, {}
testX, testY = {}, {}
timeStep = 100
for ticker in data:
    trainX[ticker], trainY[ticker] = create_dataset(trainingData[ticker], timeStep)
    testX[ticker], testY[ticker] = create_dataset(testData[ticker], timeStep)

In [42]:
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(timeStep, 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units=50))
model.add(Dropout(0.5))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [46]:
for ticker in data:
    model.fit(trainX[ticker], trainY[ticker], epochs=100, batch_size=1, verbose=1, validation_data=(testX[ticker], testY[ticker]))
    model.save(ticker + '.h5')

Epoch 1/100
25089/25089 [==============================] - 1130s 45ms/step - loss: 9.4287e-04 - accuracy: 3.9858e-05 - val_loss: 9.9872e-04 - val_accuracy: 0.0000e+00
Epoch 2/100
25089/25089 [==============================] - 1192s 48ms/step - loss: 9.3924e-04 - accuracy: 3.9858e-05 - val_loss: 0.0010 - val_accuracy: 0.0000e+00
Epoch 3/100
 2469/25089 [=>............................] - ETA: 17:19 - loss: 0.0011 - accuracy: 0.0000e+00

KeyboardInterrupt: 

In [49]:
data['VTI']

array([[7.10567730e-07],
       [7.24274116e-07],
       [7.04659823e-07],
       ...,
       [4.49080074e-06],
       [4.49080074e-06],
       [2.19003129e-01]])

In [50]:
viewDict = {}

In [ ]:
for ticker in data:
    model = load_model(ticker + '.h5')
    viewDict[ticker] = model.predict(testX[ticker])

In [52]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.black_litterman import BlackLittermanModel

In [53]:
bl = BlackLittermanModel(mu=expected_returns.mean_historical_return(data),absolute_views=viewDict,sigma=risk_models.sample_cov(data),R=None)

/home/rootdaenib/.local/lib/python3.8/site-packages/pypfopt/expected_returns.py:123: RuntimeWarning: prices are not in a dataframe
  warnings.warn("prices are not in a dataframe", RuntimeWarning)


ValueError: If using all scalar values, you must pass an index